In [ ]:
import openeo
import xarray as xr
import matplotlib.pyplot as plt
from rasterio.plot import show
# Function to download Sentinel-2 bands
def download_sentinel_band(city_name, lat, lon, start_date, end_date, bands, isSingle = False):
    connection = openeo.connect("https://openeocloud.vito.be/openeo/1.0.0").authenticate_oidc()
    cube = connection.load_collection(
        "SENTINEL2_L2A",
        spatial_extent={"west": lon[0], "east": lon[1], "south": lat[0], "north": lat[1], "crs": "EPSG:4326"},
        temporal_extent=[start_date, end_date],
        bands=bands,
        max_cloud_cover=50,
    )
    if(isSingle):
        output_file = f"{city_name}_sentinel_single.nc"
    else:
        output_file = f"{city_name}_sentinel.nc"
    cube.download(output_file)
    return output_file

# Function to plot single band data
def plot_sentinel_single_band(sentinel_band_file, band):
    ds = xr.load_dataset(sentinel_band_file)
    band_data = ds[band]  # Access the specific band data
    
    plt.figure(figsize=(10, 8))
    band_data[{"t": 1}].plot.imshow(vmin=0, vmax=2000, cmap='gray')
    plt.title(f"Sentinel-2 {band} Band")
    plt.show()

# Function to plot multi-band data
def plot_sentinel_multi_band(sentinel_band_file, bands, city, time):
    ds = xr.load_dataset(sentinel_band_file)
    band_data = ds[bands].to_array(dim="bands")
    
    plt.figure(figsize=(8, 8))
    band_data[{"t": time}].plot.imshow(vmin=0, vmax=3500)
    plt.title(f"Sentinel-2 {', '.join(bands)} Bands" + city)
    plt.show()

# List of cities with coordinates
cities = {
    #"muenchen": {"lat": [48.0800, 48.1900], "lon": [11.5000, 11.6700], "osm": "muenchen.osm.pbf", "utm_zone": 32, "sentinel_file": "muenchen_sentinel.nc", "time": 0},
    #"rotterdam": {"lat": [51.8750, 51.9700], "lon": [4.4200, 4.5400], "osm": "rotterdam.osm.pbf", "utm_zone": 31, "sentinel_file": "rotterdam_sentinel.nc", "time": 0},
    #"duesseldorf": {"lat": [51.1646, 51.3086], "lon": [6.6847, 6.8785], "osm": "duesseldorf.osm.pbf", "utm_zone": 32, "sentinel_file": "duesseldorf_sentinel.nc", "time": 0},
    #"prag": {"lat": [49.9900, 50.1100], "lon": [14.3500, 14.5400], "osm": "prag.osm.pbf", "utm_zone": 33, "sentinel_file": "prag_sentinel.nc", "time": 0},
    #"greater_london": {"lat": [51.4400, 51.5800], "lon": [-0.2000, 0.0200], "osm": "greater_london.osm.pbf", "utm_zone": 30, "sentinel_file": "greater_london_sentinel.nc", "time": 2},
    #"dublin": {"lat": [53.3244, 53.4273], "lon": [-6.3874, -6.1071], "osm": "dublin.osm.pbf", "utm_zone": 29, "sentinel_file": "dublin_sentinel.nc", "time": 1},
    #"frankfurt": {"lat": [50.0250, 50.2100], "lon": [8.5200, 8.7500], "osm": "frankfurt.osm.pbf", "utm_zone": 32, "sentinel_file": "frankfurt_sentinel.nc", "time": 2},
    #"madrid": {"lat": [40.3125, 40.6437], "lon": [-3.8890, -3.5556], "osm": "madrid.osm.pbf", "utm_zone": 30, "sentinel_file": "madrid_sentinel.nc", "time": -1},
    #"bruessel": {"lat": [50.7968, 50.9101], "lon": [4.3054, 4.4317], "osm": "bruessel.osm.pbf", "utm_zone": 31, "sentinel_file": "bruessel_sentinel.nc", "time": 0},
    "berlin": {"lat": [52.454927,52.574409], "lon": [13.294333, 13.500205], "osm": "berlin.osm.pbf", "utm_zone": 33, "sentinel_file": "berlin_sentinel.nc", "time": 1},
    #"magdeburg": {"lat": [52.0762, 52.1985], "lon": [11.5430, 11.6760], "osm": "magdeburg.osm.pbf", "utm_zone": 32, "sentinel_file": "magdeburg_sentinel.nc", "time" :0},
    #"bremerhaven": {"lat": [53.4978, 53.6008], "lon": [8.5142, 8.6552], "osm": "bremerhaven.osm.pbf", "utm_zone": 32, "sentinel_file": "bremerhaven_sentinel.nc", "time" :1},
    #"nuernberg": {"lat": [49.3955, 49.4904], "lon": [11.0023, 11.1445], "osm": "nuernberg.osm.pbf", "utm_zone": 32, "sentinel_file": "nuernberg_sentinel.nc", "time" :0},
    #"erfurt": {"lat": [50.9300, 51.0100], "lon": [11.0100, 11.1100], "osm": "erfurt.osm.pbf", "utm_zone": 32, "sentinel_file": "erfurt_sentinel.nc", "time" :0},
    #"rostock": {"lat": [54.0647, 54.1237], "lon": [12.0735, 12.1616], "osm": "rostock.osm.pbf", "utm_zone": 33, "sentinel_file": "rostock_sentinel.nc", "time" :0},
    #"chemnitz": {"lat": [50.7830, 50.8750], "lon": [12.8830, 12.9550], "osm": "chemnitz.osm.pbf", "utm_zone": 33, "sentinel_file": "chemnitz_sentinel.nc", "time" :3},
    #"potsdam": {"lat": [52.3567, 52.4348], "lon": [13.0142, 13.1208], "osm": "potsdam.osm.pbf", "utm_zone": 33, "sentinel_file": "potsdam_sentinel.nc", "time" :1},
    #"bonn": {"lat": [50.6540, 50.7640], "lon": [7.0525, 7.1650], "osm": "bonn.osm.pbf", "utm_zone": 32, "sentinel_file": "bonn_sentinel.nc", "time" :0},
    #"duisburg": {"lat": [51.3650, 51.5150], "lon": [6.6840, 6.8460], "osm": "duisburg.osm.pbf", "utm_zone": 32, "sentinel_file": "duisburg_sentinel.nc", "time" :0},
    #"osnabrueck": {"lat": [52.2450, 52.3050], "lon": [7.9500, 8.0800], "osm": "osnabrueck.osm.pbf", "utm_zone": 32, "sentinel_file": "osnabrueck_sentinel.nc", "time" :3},
}


# Define parameters for Sentinel-2 download
start_date = "2023-06-01"
end_date = "2023-06-30"
band_single = "B04"
bands_multi = [ "B04", "B03", "B02","B08"]

# Processing data for each city
for city, coords in cities.items():
    lat = coords["lat"]
    lon = coords["lon"]
    time = coords["time"]
    
    # Download and process Sentinel-2 data for single band 
    #sentinel_band_file_single = download_sentinel_band(city, lat, lon, start_date, end_date, [band_single], isSingle=True)
    #sentinel_band_file_single = f"{city}_sentinel_single.nc"
    #plot_sentinel_single_band(sentinel_band_file_single, band_single)
    
    # Download and process Sentinel-2 data for multi-band
    sentinel_band_file_multi = download_sentinel_band(city, lat, lon, start_date, end_date, bands_multi)
    #sentinel_band_file_multi = f"{city}_sentinel.nc"
    plot_sentinel_multi_band(sentinel_band_file_multi, bands_multi,city, time)


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pyrosm import OSM, get_data
import xarray as xr
import os
import matplotlib.image as mpimg
from netCDF4 import Dataset


# Function to process OSM data and save as GeoJSON
def reproject_sentinel_to_utm(sentinel_band_file, utm_zone, bands=["B02", "B03", "B04", "B08"]):
    ds = xr.load_dataset(sentinel_band_file)
    band_data = ds[bands]
    
    utm_crs = f"EPSG:{32600 + utm_zone}"  # Assuming Northern Hemisphere
    if 'crs' not in band_data.attrs:
        band_data = band_data.rio.write_crs(utm_crs, inplace=True)
    else:
        band_data = band_data.rio.reproject(utm_crs)
    
    return band_data

def process_osm_data(city_name, osm_path, sentinel_ds, bbox, output_path):
    if not os.path.exists(osm_path):
        os.makedirs(osm_path)
    fp = get_data(city_name, directory= osm_path)
    # Rename the downloaded file to the desired output path
    osm_path =f"{city}.osm.pbf"
    print(osm_path)
    # Ensure output_path is a directory
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Read OSM data using pyrosm
    #fp = get_data(city_name)
    osm = OSM(osm_path, bounding_box=bbox)
    buildings = osm.get_buildings()
    
    # Reproject OSM data to match Sentinel-2 CRS
    sentinel_crs = sentinel_ds.rio.crs
    buildings = buildings.to_crs(sentinel_crs)
    
    # Plot buildings and save as high-definition JPEG
    fig, ax = plt.subplots(figsize=(10, 10))  # Adjust the size as needed for high-definition
    buildings.plot(ax=ax, color='blue', alpha=0.6)
    
    # Remove title, axis labels, and whitespace
    ax.set_axis_off()
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    
    # Set limits to the extent of the buildings
    xmin, ymin, xmax, ymax = buildings.total_bounds
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    
    # Save the figure as a high-definition JPEG
    jpeg_output_file = f'{city_name}_buildings.jpg'
    output_file = os.path.join(output_path, jpeg_output_file)
    plt.show()
    plt.savefig(output_file, format='jpeg', dpi=300, bbox_inches='tight', pad_inches=0)  # dpi=300 for high definition
    plt.close()
    return jpeg_output_file



def plot_overlay(sentinel_band_file, sentinel_band_file_single,jpeg_path, time):
    natural_bands = ['B02', 'B03', 'B04']
    ds = sentinel_band_file
    band_data = ds[natural_bands].to_array(dim="bands")
    
    fig, ax = plt.subplots(figsize=(8, 8))
    band_data[{"t": time}].plot.imshow(ax=ax, vmin=0, vmax=3500)
    plt.title(f"Sentinel-2 Natural Color Composite ({', '.join(natural_bands)})")
    plt.show()
    
    # Read and plot the JPEG image
    img = mpimg.imread(jpeg_path)
    
    # Get the extent of the Sentinel-2 data
    xmin, ymin, xmax, ymax = band_data.rio.bounds()
    
    # Overlay the JPEG image with the correct extent
    fig, ax = plt.subplots(figsize=(8, 8))
    band_data[{"t": time}].plot.imshow(ax=ax, vmin=0, vmax=2000, cmap='gray')
    ax.imshow(img, extent=[xmin, xmax, ymin, ymax], alpha=0.6)
    
    plt.title("Overlay of Sentinel-2 Band and JPEG Image")
    plt.xlabel("x coordinate of projection [m]")
    plt.ylabel("y coordinate of projection [m]")
    plt.show()
    
    irb_bands = ["B08", "B04", "B02"]
    ds = sentinel_band_file
    band_data = ds[irb_bands].to_array(dim="bands")
    
    fig, ax = plt.subplots(figsize=(8, 8))
    band_data[{"t": time}].plot.imshow(ax=ax, vmin=0, vmax=3500)
    plt.title(f"Sentinel-2 Natural Color Composite ({', '.join(irb_bands)})")
    plt.show()
    
    single_bands = ["B04"]
    ds = sentinel_band_file_single
    band_data = ds[single_bands]
    
    fig, ax = plt.subplots(figsize=(8, 8))
    band_data[{"t": time}].plot.imshow(ax=ax, vmin=0, vmax=3500)
    plt.title(f"Sentinel-2 Natural Color Composite ({', '.join(single_bands)})")
    plt.show()
# List of cities with coordinates
cities = {
    #"muenchen": {"lat": [48.0800, 48.1900], "lon": [11.5000, 11.6700], "osm": "muenchen.osm.pbf", "utm_zone": 32, "sentinel_file": "muenchen_sentinel.nc", "time": 0},
    #"rotterdam": {"lat": [51.8750, 51.9700], "lon": [4.4200, 4.5400], "osm": "rotterdam.osm.pbf", "utm_zone": 31, "sentinel_file": "rotterdam_sentinel.nc", "time": 0},
    #"duesseldorf": {"lat": [51.1646, 51.3086], "lon": [6.6847, 6.8785], "osm": "duesseldorf.osm.pbf", "utm_zone": 32, "sentinel_file": "duesseldorf_sentinel.nc", "time": 0},
    #"prag": {"lat": [49.9900, 50.1100], "lon": [14.3500, 14.5400], "osm": "prag.osm.pbf", "utm_zone": 33, "sentinel_file": "prag_sentinel.nc", "time": 0},
    #"greater_london": {"lat": [51.4400, 51.5800], "lon": [-0.2000, 0.0200], "osm": "greater_london.osm.pbf", "utm_zone": 30, "sentinel_file": "greater_london_sentinel.nc", "time": 2},
    #"dublin": {"lat": [53.3244, 53.4273], "lon": [-6.3874, -6.1071], "osm": "dublin.osm.pbf", "utm_zone": 29, "sentinel_file": "dublin_sentinel.nc", "time": 1},
    #"frankfurt": {"lat": [50.0250, 50.2100], "lon": [8.5200, 8.7500], "osm": "frankfurt.osm.pbf", "utm_zone": 32, "sentinel_file": "frankfurt_sentinel.nc", "time": 2},
    #"madrid": {"lat": [40.3125, 40.6437], "lon": [-3.8890, -3.5556], "osm": "madrid.osm.pbf", "utm_zone": 30, "sentinel_file": "madrid_sentinel.nc", "time": -1},
    #"bruessel": {"lat": [50.7968, 50.9101], "lon": [4.3054, 4.4317], "osm": "bruessel.osm.pbf", "utm_zone": 31, "sentinel_file": "bruessel_sentinel.nc", "time": 0},
    "berlin": {"lat": [52.454927,52.574409], "lon": [13.294333, 13.500205], "osm": "berlin.osm.pbf", "utm_zone": 33, "sentinel_file": "berlin_sentinel.nc", "time": 1},
    #"magdeburg": {"lat": [52.0762, 52.1985], "lon": [11.5430, 11.6760], "osm": "magdeburg.osm.pbf", "utm_zone": 32, "sentinel_file": "magdeburg_sentinel.nc", "time" :0},
    #"bremerhaven": {"lat": [53.4978, 53.6008], "lon": [8.5142, 8.6552], "osm": "bremerhaven.osm.pbf", "utm_zone": 32, "sentinel_file": "bremerhaven_sentinel.nc", "time" :1},
    #"nuernberg": {"lat": [49.3955, 49.4904], "lon": [11.0023, 11.1445], "osm": "nuernberg.osm.pbf", "utm_zone": 32, "sentinel_file": "nuernberg_sentinel.nc", "time" :0},
    #"erfurt": {"lat": [50.9300, 51.0100], "lon": [11.0100, 11.1100], "osm": "erfurt.osm.pbf", "utm_zone": 32, "sentinel_file": "erfurt_sentinel.nc", "time" :0},
    #"rostock": {"lat": [54.0647, 54.1237], "lon": [12.0735, 12.1616], "osm": "rostock.osm.pbf", "utm_zone": 33, "sentinel_file": "rostock_sentinel.nc", "time" :0},
    #"chemnitz": {"lat": [50.7830, 50.8750], "lon": [12.8830, 12.9550], "osm": "chemnitz.osm.pbf", "utm_zone": 33, "sentinel_file": "chemnitz_sentinel.nc", "time" :3},
    #"potsdam": {"lat": [52.3567, 52.4348], "lon": [13.0142, 13.1208], "osm": "potsdam.osm.pbf", "utm_zone": 33, "sentinel_file": "potsdam_sentinel.nc", "time" :1},
    #"bonn": {"lat": [50.6540, 50.7640], "lon": [7.0525, 7.1650], "osm": "bonn.osm.pbf", "utm_zone": 32, "sentinel_file": "bonn_sentinel.nc", "time" :0},
    #"duisburg": {"lat": [51.3650, 51.5150], "lon": [6.6840, 6.8460], "osm": "duisburg.osm.pbf", "utm_zone": 32, "sentinel_file": "duisburg_sentinel.nc", "time" :0},
    #"osnabrueck": {"lat": [52.2450, 52.3050], "lon": [7.9500, 8.0800], "osm": "osnabrueck.osm.pbf", "utm_zone": 32, "sentinel_file": "osnabrueck_sentinel.nc", "time" :3},
}
# Processing data for each city
for city, data in cities.items():
    lat = data["lat"]
    lon = data["lon"]
    time = data["time"]
    bbox = [lon[0], lat[0], lon[1], lat[1]]
    bands_multi = ['B02', 'B03', 'B04','B08']
    bands_single = ["B04"]
    # Define output path
    osm_pbf_output_path_download = 'osm'
    osm_pbf_output_path = f"{city}.osm.pbf"
    geojson_output_path = f"{city}_buildings.geojson"
    sentinel_ds = reproject_sentinel_to_utm(data["sentinel_file"], data["utm_zone"], bands=bands_multi)
    sentinel_ds_single = reproject_sentinel_to_utm(f"{city}_sentinel_single.nc", data["utm_zone"], bands=bands_single)
    # Process OSM data
    jpeg_output_file = f"./output/{city}_patches/{city}_buildings.jpg"
    #plot_overlay(sentinel_ds,sentinel_ds_single ,jpeg_output_file,time)
    # Read and plot buildings
    buildings = gpd.read_file(geojson_output_path)
    buildings.plot()
    plt.title(f"Buildings in {city.replace('_', ' ').capitalize()}")
    plt.show()
